# YeaZ evaluation

Evaluates the YeaZ algorithm in a similar manner as the Mask-RCNN pipeline.

We've precomputed the output of YeaZ for the YIT data and it's downloaded as `YeaZ_results.zip`.

In [8]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
%matplotlib notebook

try:
  from download import download
except ImportError:
  !pip3 install download
  from download import download

try:
    # raise ImportError()
    from yeastcells import data, evaluation
except ImportError:
    !test -e yeastcells-detection-maskrcnn || git clone https://github.com/ymzayek/yeastcells-detection-maskrcnn.git
    !cd yeastcells-detection-maskrcnn; git pull origin main
    !pip3 install ./yeastcells-detection-maskrcnn
    from yeastcells import data, evaluation

## Downloads

Downloads ground truth and YeaZ results

In [9]:
pipeline_path = f'/content/yeastcells-detection-maskrcnn'

download(
    'https://datascience.web.rug.nl/YeaZ_results.zip',
    f'{pipeline_path}/YeaZ_results/YeaZ_results.zip'
)

download(
    'https://datascience.web.rug.nl/YIT-Benchmark2.zip', 
    f'{pipeline_path}/YIT-Benchmark2/YIT-Benchmark2.zip'
)

!cd '{pipeline_path}' && unzip -o 'YIT-Benchmark2/YIT-Benchmark2.zip' -d YIT-Benchmark2
!cd '{pipeline_path}' && unzip -o 'YeaZ_results/YeaZ_results.zip' -d YeaZ_results

Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
Replace is False and data exists, so doing nothing. Use replace=True to re-download the data.
Archive:  YIT-Benchmark2/YIT-Benchmark2.zip
  inflating: YIT-Benchmark2/TestSet10/description.txt  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/GroundTruth_Segmentation.csv  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/GroundTruth_Tracking.csv  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/Segmentation/SegGT_frame001.png  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/Segmentation/SegGT_frame002.png  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/Segmentation/SegGT_frame003.png  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/Segmentation/SegGT_frame004.png  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/Segmentation/SegGT_frame005.png  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/Segmentation/SegGT_frame006.png  
  inflating: YIT-Benchmark2/TestSet10/GroundTruth/

## YeaZ results

Prepares the YeaZ results for evaluation

In [22]:
def load_test_yeaz_detections(testset_name = 'TestSet1'):
  # one mask per frame, with integer encoded segmentation.
  masks = data.read_tiff_mask(f'{pipeline_path}/YeaZ_results/YeaZ_masks_{testset_name}.tiff')

  # a dataframe with every detection per frame seperately
  yeaz_detections = pd.DataFrame([
    {'frame': frame, 'cell': cell}
    for frame, mask in enumerate(masks)
    for cell in np.unique(mask)
    if cell > 0
  ])

  # create one-hot encoded masks, like we get from the Mask-RCNN, such that they
  # fit our evaluation pipeline.
  masks = (
      masks[yeaz_detections['frame']] # repeat frames per detection of a frame
      == yeaz_detections['cell'].values[:, None, None] # one-hot encode
  )
  # we could use the dataframe's index, but a seperate column is safer to mutations
  yeaz_detections['mask'] = np.arange(len(masks))
  return masks, yeaz_detections

## Ground truth

The ground truth consists of annotations of
 * `frame`: frame number starting at 1,
 * `cell`: tracks cells over frames, and
 * `x`, `y`: cell position.

In [20]:
def get_ground_truth(testset_name = 'TestSet1'):
  ground_truth = pd.read_csv(
      f'{pipeline_path}/YIT-Benchmark2/{testset_name}/GroundTruth/GroundTruth_Segmentation.csv',
      sep=', ' # there's an awkward space next to the commas
  ).drop('Cell_colour', axis=1).rename(
      {'Frame_number': 'frame',	'Cell_number': 'cell',
      'Position_X' :'x', 'Position_Y': 'y'}, axis=1).reindex()

  # detections start counting at 0, ground truths at 1. rectify:
  ground_truth['frame'] = ground_truth['frame'] - 1
  return ground_truth

## Evaluation

Evaluates the YeaZ output similar to the Mask-RCNN

In [16]:
testset_names = [
 'TestSet1',
 'TestSet2',
 'TestSet3',
 'TestSet4',
 'TestSet5',
 'TestSet6',
 'TestSet7',
#  'TestSet8',
#  'TestSet9',
#  'TestSet10',
]

### Results

In [30]:
metrics = []

for testset_name in tqdm(testset_names, desc='Overall evaluation progress'):
  masks, yeaz_detections = load_test_yeaz_detections(testset_name)
  ground_truth = get_ground_truth(testset_name)
  segmentation_metrics = evaluation.calculate_metrics(
      evaluation.get_segmention_metrics(ground_truth, yeaz_detections, masks))
  tracking_metrics = evaluation.calculate_metrics(
      evaluation.get_tracking_metrics(ground_truth, yeaz_detections, masks))

  metrics.extend([
      {'test set': testset_name, 'task': task, 'metric': metric, 'value': value}
      for task, result in {
          'tracking': tracking_metrics,
          'segmentation': segmentation_metrics}.items()
      for metric, value in result.items()
  ])
results = pd.DataFrame(metrics)

results.pivot(('test set', 'task'), columns=['metric'], values='value')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after rem

metric                 Accuracy  F1-score  Precision    Recall
test set task                                                 
TestSet1 segmentation  0.646002  0.784935   0.665672  0.956261
         tracking      0.666061  0.799563   0.682609  0.964881
TestSet2 segmentation  0.466443  0.636156   0.476027  0.958621
         tracking      0.490706  0.658354   0.503817  0.949640
TestSet3 segmentation  0.857882  0.923505   0.903943  0.943933
         tracking      0.847113  0.917229   0.888066  0.948372
TestSet4 segmentation  0.836032  0.910694   0.944602  0.879137
         tracking      0.677147  0.807498   0.820978  0.794454
TestSet5 segmentation  0.779333  0.875983   0.907735  0.846378
         tracking      0.414964  0.586537   0.595339  0.577991
TestSet6 segmentation  0.882609  0.937644   0.957547  0.918552
         tracking      0.877698  0.934866   0.938462  0.931298
TestSet7 segmentation  0.927757  0.962525   0.975350  0.950032
         tracking      0.761438  0.864564   0.870703  0.858511

### Download

In [31]:
results.pivot(('test set', 'task'), columns=['metric'], values='value').to_excel('yeaz-performance.xlsx')

try:
  from google.colab.files import download
  download('yeaz-performance.xlsx')
except ImportError:
  print('Not on Google Colab, won\'t start download')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>